In [2]:
import numpy as np
import pandas as pd

from catalyst.data.sampler import BalanceClassSampler
from sklearn.model_selection import StratifiedKFold
from torch.functional import istft

In [3]:
df = pd.read_csv("../data/merged.csv")
df_20_3 = df.loc[(df["source"] == 2020) & (df["label"] == 3)].copy()
df_20_3 = df_20_3.sample(frac=0.7)
# df_20 = df.loc[df["source"] == 2020]
# df_20["data_dir"] = "train_images"
# df_19_0 = df.loc[(df["source"] == 2019) & (df["label"] == 0)]
# df_19_0["data_dir"] = "train/cbb/"
# df_19_2 = df.loc[(df["source"] == 2019) & (df["label"] == 2)]
# df_19_2["data_dir"] = "train/cgm/"
# df_19_4 = df.loc[(df["source"] == 2019) & (df["label"] == 4)]
# df_19_4["data_dir"] = "train/healthy/"

# df = pd.concat([df_20, df_19_0, df_19_2, df_19_4], axis=0).reset_index(drop=True)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [27]:
df["kfold"] = -1
df = df.sample(frac=1).reset_index(drop=True)
y = df["label"].values
skf = StratifiedKFold(n_splits=5)

for (fold_num), (train_index, val_index) in enumerate(skf.split(X=df, y=y)):
    df.loc[df.iloc[val_index].index, "kfold"] = fold_num

train_df = df.loc[df["kfold"] != 0].reset_index(drop=True)
valid_df = df.loc[df["kfold"] == 0].reset_index(drop=True)


In [9]:
df = pd.read_csv("../data/merged.csv")
df = df.drop(["label"], axis=1)

df_label = pd.read_csv(f"../good_result/resnext101_32x8d_exp001/soft_labels.csv")

df = pd.merge(df, df_label, on="image_id", how="inner")
df_20 = df.loc[df["source"] == 2020]
df_20["data_dir"] = "train_images"
df_19_0 = df.loc[(df["source"] == 2019) & (df["label"] == 0)]
df_19_0["data_dir"] = "train/cbb/"
df_19_2 = df.loc[(df["source"] == 2019) & (df["label"] == 2)]
df_19_2["data_dir"] = "train/cgm/"
df_19_4 = df.loc[(df["source"] == 2019) & (df["label"] == 4)]
df_19_4["data_dir"] = "train/healthy/"

df = pd.concat([df_20, df_19_0, df_19_2, df_19_4], axis=0).reset_index(drop=True)

In [10]:
df



,image_id,source,label,CBB,CBSD,CGM,CMD,Healthy,data_dir
0,1000015157.jpg,2020,0,0.500678,0.166694,0.166937,0.047325,0.118366,train_images
1,1000201771.jpg,2020,3,0.044552,0.044911,0.054865,0.808968,0.046703,train_images
2,100042118.jpg,2020,1,0.050457,0.114405,0.041060,0.072762,0.721316,train_images
3,1000723321.jpg,2020,1,0.036631,0.747164,0.039886,0.111257,0.065063,train_images
4,1000812911.jpg,2020,3,0.054205,0.055162,0.065534,0.774501,0.050598,train_images
...,...,...,...,...,...,...,...,...,...
22741,train-healthy-82.jpg,2019,4,0.126412,0.045210,0.045921,0.047253,0.735204,train/healthy/
22742,train-healthy-163.jpg,2019,4,0.060482,0.050668,0.042054,0.077575,0.769220,train/healthy/
22743,train-healthy-189.jpg,2019,4,0.031730,0.035799,0.036658,0.060903,0.834910,train/healthy/
22744,train-healthy-243.jpg,2019,4,0.049920,0.030043,0.089280,0.067524,0.763233,train/healthy/


In [13]:
df.iloc[1, 3:8].values

array([0.04455166682600975, 0.044911496341228485, 0.05486523360013962,
       0.8089684247970581, 0.046703297644853585], dtype=object)

In [36]:
tmp_df.label.value_counts()

3    10526
2     4828
4     4624
0     3579
1     3502
Name: label, dtype: int64

In [59]:
import torch
import torch.nn as nn
import numpy as np
class CrossEntropyLossOneHot(nn.Module):
    def __init__(self):
        super(CrossEntropyLossOneHot, self).__init__()
        self.log_softmax = nn.LogSoftmax(dim=-1)

    def forward(self, preds, labels):
        return torch.mean(torch.sum(-labels * self.log_softmax(preds), -1))

class CustomeLoss(nn.Module):
    def __init__(self, l=0.9, t=1.0):
        super(CustomeLoss, self).__init__()
        self.l = l
        self.t = t
        self.criterion = CrossEntropyLossOneHot()

    def forward(self, yt_soft, ys_soft, y_true):
        ys_hard = torch.nn.functional.one_hot(yt_soft.argmax(1), num_classes=5)
        loss = (1-self.l) * self.criterion(yt_soft, ys_soft) + self.l * self.t * self.t * self.criterion(y_true, ys_hard)

        return loss

In [60]:
criterion = CustomeLoss(l=0.9, t=1)

In [65]:
yt_soft = torch.tensor(np.random.random((2, 5)))
ys_soft = torch.tensor(np.random.random((2, 5)))
t_true = torch.tensor([[0, 0, 1, 0, 0], [1, 0, 0, 0, 0]], dtype=torch.float)

In [66]:
criterion(yt_soft, ys_soft, t_true)

tensor(1.6403, dtype=torch.float64)

In [51]:
torch.zeros_like(yt_soft).scatter_(0, yt_soft.argmax(1), 5)

RuntimeError: Index tensor must have the same number of dimensions as self tensor

In [50]:
yt_soft

tensor([[0.9302, 0.2655, 0.6107, 0.1034, 0.4650],
        [0.3501, 0.4341, 0.3029, 0.1930, 0.9354]], dtype=torch.float64)

In [52]:
ys_hard = torch.nn.functional.one_hot(yt_soft.argmax(1), num_classes=5)

tensor([[1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1]])

In [64]:
c = CrossEntropyLossOneHot()
c(yt_soft, ys_soft)
ys_hard = torch.nn.functional.one_hot(yt_soft.argmax(1), num_classes=5)
c(t_true, ys_hard)

RuntimeError: "log_softmax_lastdim_kernel_impl" not implemented for 'Long'